<a href="https://colab.research.google.com/github/comparativechrono/nanopore-methylation/blob/main/Nanopore_NGS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finding sequences that match a query in nanopore long read fastq files

##Set-up

When you start a colab session you are starting a new virtual machine. If we want to use notebooks for reproducible research we need to be able to set up all the programs and import all the data we need for our analysis each time we begin. This might seem like a lot of work - but it is worth doing. Why? Because it makes our research reproducible. There are a couple of things we need to do. We will set up conda as it is the best way to use tools, and then we will use conda to install the programs needed. This may take around 10 minutes. The code cells are hidden.

In [2]:
#@title Set up conda on notebook
%%capture
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh 
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh 
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local 
import sys 
sys.path.append('/usr/local/lib/python3.7/site-packages/')


In [3]:
#@title Install the programs you need
%%capture
# FastQC is a program designed to spot potential problems in high throughput sequencing datasets. 
!conda install -c bioconda samtools -y
# Multiqc can aggregate and summarize all the QC data and alignment log data in one file 
!pip install multiqc
# BLAST - the BLAST algorithm 
! conda install -c bioconda blast -y
# Magic-blast - BLAST for high-throughput sequencing reads 
! conda install -c bioconda magicblast -y

##The problem:
We want to perform the following tasks:
1.	Convert fastq to fasta 
2.	Make a fasta file containing the GOI (i.e. the 2kb of the promoter)
3.	Use fasta file as a BLAST library
4.	Use GOI as the sequence to search
5.	Use BLAST to identify reads containing the sequence of interest
6.	Use the ID of the read to cross-reference to the bam file
7.	Identify the CIGAR string in the bam file

This notebook takes you up to step 5. 

## The solution:
We need to convert the FASTQ file into something that can be searched against efficiently using an algorithm. We will do this using BLAST. We need to choose a way to transform our FASTQ data into FASTA data. Then we need to make a BLAST library from our sequence FASTA data and search it using the sequence of our GOI.

In [4]:
# as this notebook is written for colab we will cheat and bring in a fastq.gz file using URL
! wget https://www.dropbox.com/s/yydq3i13t2zu0gg/FAT50823_pass_d652af8d_0.fastq.gz?dl=1 -O example.fastq.gz

--2022-05-31 13:03:42--  https://www.dropbox.com/s/yydq3i13t2zu0gg/FAT50823_pass_d652af8d_0.fastq.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.7.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.7.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/yydq3i13t2zu0gg/FAT50823_pass_d652af8d_0.fastq.gz [following]
--2022-05-31 13:03:43--  https://www.dropbox.com/s/dl/yydq3i13t2zu0gg/FAT50823_pass_d652af8d_0.fastq.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc40ba017df868f50d87a3d69be0.dl.dropboxusercontent.com/cd/0/get/BmVgV0UdaUpHSyfUicQgwaam4FRkjFIFqDnpP8HLJm6BstTI265MNg43AtCswX_rJCDuv1fVLhWzif7JWnqMbjzaztPXikQZSFn60q5QlJT16mXDWq8LHgC9OXAx1zMoBRoYxK0Giy2xKHrtv0C7yytIXbm-k0PUd8xrSgr8gylYHJuKt1kjT9u1seam2mTNMLI/file?dl=1# [following]
--2022-05-31 13:03:43--  https://uc40ba017df868f50d87a3d69be0.dl.dropboxusercon

**Step 1)** Get fastq files unzipped and merged to be further processed

In [5]:
# extract the fastq files using gunzip recursively
! gunzip *.gz

gzip: example.fastq already exists; do you wish to overwrite (y or n)? y


In [6]:
# merge the fastq files together by concatanating them
!cat *.fastq > bigfile.fastq

**Step 2)** Convert fastq to fasta to be BLAST input. There are many ways to do this, I like using `awk`

In [7]:
%%bash
cat bigfile.fastq | awk '{if(NR%4==1) {printf(">%s\n",substr($0,2));} else if(NR%4==2) print;}' >  output.fasta
head output.fasta

>8353186d-7240-49f1-b8a6-4120e21ae2dc runid=d652af8d03078163af0aadbe074b35c7cfcc6dc6 read=19 ch=434 start_time=2022-05-25T17:08:06.029807+01:00 flow_cell_id=FAT50823 protocol_group_id=220525_ZF_ZT9_LD sample_id=ZF_ZT9_LD parent_read_id=8353186d-7240-49f1-b8a6-4120e21ae2dc basecall_model_version_id=unknown
ATCAGATTTGGGTGTTTAACCGTTTCCCCTTTATCGTGGCTGCTTTCGCGTTTTCGTGCGCCCTTCAGTGTGTAGTCTTTTTATCCCAGCATCCCTCTGGCCATTGTGAGCATCTGCATAATGCATGATCAATATGTTGGTTTATAAAGGGCTCAATCTGACAGCGAAGCATACAGTCATATATTCACGTCTGATCCTGTGGATGTGGAGTGGATGGAGCGGGCCTGTGGCAAGATGCGCAGTGTCAGCTTTCACTTACTCACTTAATCCAGGCCCACCCTTCAACCTCATCCAGAGAGCTATGGTTAGTTGAGAGGGAAGAAAAACTGACTTTAATCAGCTCTGGCTTCAGGAAACTCAGCAGTGACGAGGATTGGAGAATTTAATGATGAGCTCAGTGGAAGTTGAAGATATTGT
>27b8a01d-a923-4e49-8eed-69ebf5738756 runid=d652af8d03078163af0aadbe074b35c7cfcc6dc6 read=21 ch=282 start_time=2022-05-25T17:08:06.029807+01:00 flow_cell_id=FAT50823 protocol_group_id=220525_ZF_ZT9_LD sample_id=ZF_ZT9_LD parent_read_id=27b8a01d-a923-4e49-8eed-69ebf5738756 ba

**Step 3)** Make a blast library. magicblast has a command to make a blast database <code>makeblastdb</code>. Note that some older information on the internet will recommend using the EMBOSS formatdb package. Please do not use this. There is a good overview here: https://www.ncbi.nlm.nih.gov/books/NBK569841/. Detailed instructions for using <code>makeblastdb</code> are available here: https://ncbi.github.io/magicblast/cook/blastdb.html

In [8]:
%%!
makeblastdb -in output.fasta -dbtype nucl -out database

['',
 '',
 'Building a new DB, current time: 05/31/2022 13:33:38',
 'New DB name:   /content/database',
 'New DB title:  output.fasta',
 'Sequence type: Nucleotide',
 'Deleted existing Nucleotide BLAST database named /content/database',
 'Keep MBits: T',
 'Maximum file size: 1000000000B',
 'Adding sequences from FASTA; added 4000 sequences in 0.416106 seconds.']

For this command we specify the following arguments:
        <code>-in </code>
        <code>-dbtype</code>
        <code>-out</code>

**Step 4)** To run BLAST we need a query sequence. This will be your GOI. There will be many ways that you can get this. For convenience sake here we will use a generic input form, but replace the following code cell with your seqeunce.

In [1]:
#@title Input your fasta file here

query_name = "scn12aa" #@param {type:"string"}
fasta_query = "gatcaatgttttgcggattgcatatgaaggggagacgacacctgtaatgaaaagaaaagggaatcccgccgtttttatatttatttgaagtgttttcatgcctaaacaaagcgaaagcacagaacagactcgcacttaagagcaagaggcgacccttcggcggtatgggttgcatacagggatatgggttgcatacggtttgttacccttcagagaaaaactaaaaatatgggaaaatacctttatgggatgatagcaaatcatcagaggatagaaatgtaaaatacgggagaatcccaggaaaaatgggagggttgagaggtatgAttagaatccaaaatcgaaattgaattttcgattaattgcacagcgctaAATACAACACTTATCAAAAAGGTTTAAAAAGCACCCTAGTGTGAACTTAAAGAAGTGGTTTGCTTGCAAATTTAAACTATAACCCTTTTGACATCCTAGAGACACAAAATTAAGAATAATCCAGCCTcacacacacatacacacacacacacacacacatactgcttgcttcgtaatgtgggattacATAAGCAGCACTATTGGATTGTGCTGGCACCTTCAGGCCAGTAATCTCCCTAAATCATTTTAAATATGTTGCCATTACATGCAGGGAAAATCTCATCAAAATTGTATTAAGGCATGCCTATTGCTGATGCAGAGAAAATGAAACAAGGAGAATGAGTGTGTATTCAGGCAGATTTAGAACAGCTCTCTCTCTCGCCGTCTCAGATTATACTGGAGGTCACTGACCTTGACTTCATGGATTAGATGCATATGCTAGTATAAGGCCATATGTATCTGCAATGCATGCATTCTATTAGCGGCCTGTGATGTTATCATGCCAAACAAAGAGGATTTTTCAATCAATGTTGATTTCCTGGATTATTTGAAATGTTATACACAAACGAATGTTCAAATTACTTAATTAGCATGATATATAATACTAGAGTATATGGGTCCAAGCGGCTAAATGATAACATGCATTACA" #@param {type:"string"}

sequences = {query_name:fasta_query}
output_path = 'input.fa'
output_file = open(output_path,'w')

identifier_line = ">" + query_name + "\n"
output_file.write(identifier_line)
sequence_line = fasta_query + "\n"
output_file.write(sequence_line)
    
#Close the file when we're done
output_file.close()

In [11]:
# check your fast a file
! head input.fa

head: cannot open 'output.fa' for reading: No such file or directory


**Step 5)** When you have created a BLAST library and formatted your query seqeunce it is time to run blastn using one of the query sequences in the FASTA query folder. We use blastn from the Blast+ package. The overview in the BLAST+ user manual is useful to read before beginning: https://www.ncbi.nlm.nih.gov/books/NBK569856/.

In [9]:
%%!
blastn -db database -query input.fa -out results.out

[]

Now we use grep to find our sequence in the fasta file matching the header name in the blast ouput file. You will have to replace the name after -w with the name you find.

In [13]:
! grep -A 1 -w "13dfea90-b327-48cd-8fdd-d367a03ae674 runid=d652af8d03078163af0aadbe074b35c7cfcc6dc6" output.fasta > my.fasta